In [ ]:
import pathlib as Path
import sys
import polars as pl
import numpy as np
import pandas as pd
import time
import copy
import matplotlib.pyplot as plt
import altair as alt
import hvplot.polars as hvpl 
import seaborn as sns
import autoviz as av
import sweetviz as sv
import polars.selectors as cs
import altair as alt
root = Path.Path('/home/georgios/Coding-Projects/Expedia_Personalized/expedia-personalized-recommendations').resolve()
sys.path.append(str(root))
from src.features.dtypes import downcast_dtypes
alt.data_transformers.enable("vegafusion")





In [67]:
# SET THE ROOT DIRECTORY
root = Path.Path('/home/georgios/Coding-Projects/Expedia_Personalized/expedia-personalized-recommendations').resolve()
sys.path.append(str(root))
print(root)
RAW_DIR = Path.Path('data/raw')
FEATURES_DIR= Path.Path(root/'src/features')
train_df = pl.read_csv(root / RAW_DIR / 'train.csv', null_values="NULL")
train_df=train_df.with_columns(date_time=train_df["date_time"].str.to_datetime(format="%Y-%m-%d %H:%M:%S"))
train_df=downcast_dtypes(train_df)

/home/georgios/Coding-Projects/Expedia_Personalized/expedia-personalized-recommendations
Downcasting data types...
Column 'srch_id' casted to Int32
Column 'site_id' casted to Int8
Column 'visitor_location_country_id' casted to Int16
Column 'prop_country_id' casted to Int16
Column 'prop_id' casted to Int32
Column 'prop_starrating' casted to Int8
Column 'prop_review_score' casted to Float32
Column 'prop_brand_bool' casted to Int8
Column 'prop_location_score1' casted to Float32
Column 'prop_location_score2' casted to Float32
Column 'prop_log_historical_price' casted to Float32
Column 'position' casted to Int8
Column 'price_usd' casted to Float32
Column 'promotion_flag' casted to Int8
Column 'srch_destination_id' casted to Int16
Column 'srch_length_of_stay' casted to Int8
Column 'srch_booking_window' casted to Int16
Column 'srch_adults_count' casted to Int8
Column 'srch_children_count' casted to Int8
Column 'srch_room_count' casted to Int8
Column 'srch_saturday_night_bool' casted to Int8
C

In [ ]:


data=train_df.select(cs.matches("usd"))
data.describe()

In [80]:
import polars as pl

def detect_outliers_modified_zscore(df: pl.LazyFrame, col: str, threshold=3.5) -> pl.LazyFrame:
    """
    Detects outliers using the Modified Z-Score (MAD) method in a LazyFrame.
    
    - Replaces outliers with `None` in the same column.
    - Adds a separate `_mad_outlier_flag` column to indicate outliers.

    Args:
        df (pl.LazyFrame): The input LazyFrame.
        col (str): The column name to check for outliers.
        threshold (float): The MAD threshold for outlier detection.

    Returns:
        pl.LazyFrame: Updated LazyFrame with the column modified and an outlier flag.
    """

    # Compute MAD-based modified z-score
    mad_score = 0.6745 * (pl.col(col) - pl.col(col).median()) / pl.col(col).abs().median()

    # Boolean flag for outliers
    mad_flag = mad_score.abs() > threshold

    # Count outliers (force computation with collect)
    outlier_count = df.select(mad_flag.sum()).collect().item(0, 0)

    print(f"🔹 Modified Z-Score (MAD) Method: Found {outlier_count} outliers in '{col}'.")
    print(f"Replacing outliers with None and saving flags in '{col}_mad_outlier_flag'.")

    # Modify the LazyFrame without collecting
    df = df.with_columns(
        pl.when(mad_flag).then(None).otherwise(pl.col(col)).alias(f"{col}_mad_filtered")
    )

    return df  # ✅ Still a LazyFrame (no collect)


train_df = detect_outliers_modified_zscore(pl.LazyFrame(train_df), "price_usd", threshold=3.5)

#plot the original distribution vs the distribution after removing outliers



🔹 Modified Z-Score (MAD) Method: Found 51768 outliers in 'price_usd'.
Replacing outliers with None and saving flags in 'price_usd_mad_outlier_flag'.


In [ ]:

def plot_conditional_barchart(df: pl.DataFrame, var_x: str, click=False) -> alt.Chart:
    """
    Generate an interactive Altair bar plot for categorical variables with Clicked vs Not Clicked.

    Args:
        df (pl.DataFrame): The input dataframe containing the required columns.
        var_x (str): The categorical variable for the X-axis (e.g., "prop_starrating").
        click (bool): If True, filters only Clicked data.
    
    Returns:
        alt.Chart: The interactive Altair visualization.
    """

    # # ✅ Apply Clicked Filter if `click=True`
    # if click:
    #     df = df.filter(pl.col("click_bool") == 1)

    # ✅ Compute proportions dynamically
    n_rows = df.shape[0]  
    df = (
        df.group_by([var_x, "click_bool", "booking_bool"])
        .agg(((pl.count() / n_rows) * 100).alias("proportion"))
        .sort("proportion", descending=True)
    )

    print(f"Sanity Check: {df.shape[0]} rows in aggregated DataFrame. \n Sum of proportions: {df['proportion'].sum()}")

    # ✅ Multi-Selection Dropdown for Clicked vs Not Clicked
    click_selection = alt.param(
        name="click_bool_selection",
        bind=alt.binding_select(
            options=["All", 0, 1],
            labels=["Both", "Not Clicked", "Clicked"],
            name="Click Status"
        ),
        value="All"
    )

    # ✅ Multi-Selection Dropdown for Booked vs Not Booked
    book_selection = alt.param(
        name="book_bool_selection",
        bind=alt.binding_select(
            options=["All", 0, 1],
            labels=["Both", "Not Booked", "Booked"],
            name="Booking Status"
        ),
        value="All"
    )

    # ✅ Combined Filtering Logic
    filter_condition = (
        ((click_selection == "All") | 
         ((click_selection == 0) & (alt.datum.click_bool == 0)) |
         ((click_selection == 1) & (alt.datum.click_bool == 1))) &
        ((book_selection == "All") | 
         ((book_selection == 0) & (alt.datum.booking_bool == 0)) |
         ((book_selection == 1) & (alt.datum.booking_bool == 1)))
    )

    # ✅ Define Y-Axis
    y_axis = alt.Y(
        "proportion:Q",
        title=f"Percentage of {var_x.replace('_', ' ').title()}",
        scale=alt.Scale(domain=[0, 100]),
        axis=alt.Axis(tickCount=10)
    )

    # ✅ Enable Zooming
    zoom = alt.selection_interval(bind="scales")

    # ✅ Dynamic Title Based on Selection
    title_expr = alt.expr(
        f"click_bool_selection === 1 ? 'Distribution of {var_x.replace('_', ' ').title()} for Clicked Users' : "
        f"click_bool_selection === 0 ? 'Distribution of {var_x.replace('_', ' ').title()} for Not Clicked Users' : "
        f"'Distribution of {var_x.replace('_', ' ').title()} by Click & Booking Status'"
    )

    # ✅ Create the Chart
    chart = alt.Chart(df).transform_calculate(
        click_label="datum.click_bool == 1 ? 'Clicked' : 'Not Clicked'",
        book_label="datum.booking_bool == 1 ? 'Booked' : 'Not Booked'"
    ).mark_bar(opacity=0.8).encode(
        x=alt.X(f"{var_x}:O", title=var_x.replace('_', ' ').title()),
        xOffset=alt.XOffset("book_label:N"),
        y=y_axis,
        color=alt.Color(
            "book_label:N",
            title="Booking Status",
            scale=alt.Scale(domain=["Not Booked", "Booked"], range=["#d62728", "#1f77b4"]),
        ),
        tooltip=[alt.Tooltip(var_x, title=var_x.replace("_", " ").title()), 
                 alt.Tooltip("proportion", title="Percentage"),
                 alt.Tooltip("click_label:N", title="Click Status"), 
                 alt.Tooltip("book_label:N", title="Booking Status")]
    ).transform_filter(
        filter_condition
    ).add_params(
        click_selection,
        book_selection,
        zoom
    ).properties(
        width=800,
        height=400,
        title=alt.TitleParams(
            text=title_expr,
            fontSize=14,
            anchor="middle",
            fontWeight="bold"
        )
    )

    return chart


plot_conditional_barchart(train_df, "booking_day_of_week", click=False)


In [41]:

def plot_timeseries(df: pl.DataFrame, datetime_col: str = "date") -> alt.Chart:
    """
    Generate an interactive daily time series plot for Clicked vs. Booked counts with zooming & panning.

    Args:
        df (pl.DataFrame): The input dataframe containing datetime, click_bool, and booking_bool.
        datetime_col (str): The name of the datetime column.

    Returns:
        alt.Chart: The interactive Altair visualization.
    """
    # ✅ Ensure the Date Column Exists
    if datetime_col not in df.columns:
        print(f"Column '{datetime_col}' not found. Extracting date from any time column...")
        df = df.with_columns(
            pl.col("^.*time.*$").dt.date().alias(datetime_col)  # Extract date from any time column
        )

    # ✅ Aggregate Daily Clicks & Bookings
    df = (
        df.group_by(datetime_col)
        .agg(
            pl.col("click_bool").sum().alias("click_count"),
            pl.col("booking_bool").sum().alias("book_count")
        )
        .sort(datetime_col)
    )

    # ✅ Convert to Long Format for Altair
    melted_df = df.melt(id_vars=[datetime_col], variable_name="metric", value_name="count")

    # ✅ Get First and Last Dates for Proper Axis Scaling
    first_date, last_date = melted_df[datetime_col].min(), melted_df[datetime_col].max()

    # ✅ Dropdown for Selecting Clicked vs Booked
    selection_param = alt.param(
        name="selection",
        bind=alt.binding_select(
            options=["Both", "click_count", "book_count"],
            labels=["Both", "Clicked", "Booked"],
            name="Filter"
        ),
        value="Both"
    )

    # ✅ Filtering Logic
    filter_condition = (
        (selection_param == "Both") | 
        ((selection_param == "click_count") & (alt.datum.metric == "click_count")) | 
        ((selection_param == "book_count") & (alt.datum.metric == "book_count"))
    )

    # ✅ Dynamic Title Expression
    title_expr = alt.expr(
        "selection === 'click_count' ? 'Daily Click Count' : "
        "selection === 'book_count' ? 'Daily Booking Count' : "
        "'Daily Clicked vs Booked Count'"
    )

    # ✅ Enable Zooming & Panning
    zoom = alt.selection_interval(bind="scales")

    # ✅ Create the Time Series Chart
    chart = alt.Chart(melted_df).mark_line(point=True).encode(
        x=alt.X(
            f"{datetime_col}:T",
            title="Date",
            scale=alt.Scale(domain=[first_date, last_date])  # Ensure the axis starts from first to last date
        ),
        y=alt.Y("count:Q", title="Count"),
        color=alt.Color("metric:N", title="Metric", scale=alt.Scale(domain=["click_count", "book_count"], range=["#1f77b4", "#ff7f0e"])),
        tooltip=[alt.Tooltip(datetime_col, title="Date"), alt.Tooltip("metric:N", title="Metric"), alt.Tooltip("count:Q", title="Count")]
    ).transform_filter(
        filter_condition
    ).add_params(
        selection_param,
        zoom  # ✅ Add zooming & panning
    ).properties(
        width=800,
        height=400,
        title=alt.TitleParams(
            text=title_expr,
            fontSize=14,
            anchor="middle",
            fontWeight="bold"
        )
    ).configure_axisX(
        labelAngle=45
    )

    return chart
# ✅ Generate the Time Series Plot
plot_timeseries(train_df, "booking_date")


Column 'booking_date' not found. Extracting date from any time column...


TypeError: 'LazyFrame' object is not subscriptable (aside from slicing)

Use `select()` or `filter()` instead.

In [ ]:
import polars as pl
import altair as alt
import numpy as np
from typing import Optional

def plot_histogram_optional_outliers(
    df: pl.DataFrame,
    numeric_col: str,
    outlier_col: Optional[str] = None,
    bins: int = 30
) -> alt.Chart:
    """
    Plot a histogram of `numeric_col`. If `outlier_col` is provided (and exists),
    create a dropdown param with "All" / "NoOutliers". If user picks "NoOutliers",
    we filter out rows where outlier_col == True.

    Args:
        df (pl.DataFrame): Eager Polars DataFrame with the numeric column.
        numeric_col (str): The name of the numeric column to bin & plot.
        outlier_col (Optional[str]): The name of the boolean column that marks outliers 
                                     (True => outlier). If None or missing, no toggle is displayed.
        bins (int): Max number of bins for Altair.

    Returns:
        alt.Chart: An Altair histogram that optionally toggles outliers on/off.
    """

    # Convert to Arrow
    arrow_data = df.to_arrow()

    # 1) If outlier_col is present in df, define the param & filter logic.
    if outlier_col and outlier_col in df.columns:
        outlier_param = alt.param(
            name="outlier_mode",
            bind=alt.binding_select(
                options=["All", "NoOutliers"],
                labels=["All", "NoOutliers"],
                name="Mode"
            ),
            value="All"  # default
        )

        # Filter condition: if user picks "NoOutliers", require outlier_col == false. Otherwise, show all.
        filter_condition = alt.expr.if_(
            outlier_param == "NoOutliers",
            alt.datum[outlier_col] == False,
            True  # if "All"
        )

        # Dynamic title expression
        title_expr = alt.expr(
            f"outlier_mode === 'NoOutliers' ? 'Histogram ({numeric_col} Without Outliers)' : "
            f"'Histogram ({numeric_col} All)'"
        )

        # Build chart
        chart = (
            alt.Chart(arrow_data)
            .mark_bar(opacity=0.6)
            .transform_filter(filter_condition)
            .encode(
                x=alt.X(
                    f"{numeric_col}:Q",
                    bin=alt.Bin(maxbins=bins),
                    title=numeric_col
                ),
                y=alt.Y("count()", title="Frequency"),
                color=alt.Color(
                    # color by outlier col (optional), or just a single color
                    # but let's highlight outliers in a different color
                    outlier_col,
                    title="Is Outlier?",
                    scale=alt.Scale(domain=["false","true"], range=["#1f77b4","#d62728"])
                ),
                tooltip=[
                    alt.Tooltip(f"{numeric_col}:Q", title=numeric_col),
                    alt.Tooltip(f"{outlier_col}:N", title="Outlier?")
                ]
            )
            .add_params(outlier_param)
            .properties(
                width=600,
                height=400,
                title=alt.TitleParams(
                    text=title_expr,
                    fontSize=14,
                    anchor="middle",
                    fontWeight="bold"
                )
            )
            .configure_axisX(labelAngle=45)
        )

    else:
        # 2) If outlier_col is None or doesn't exist, no toggle (just a single chart).
        #    We can still color by outlier col if you want, but we skip the param if it doesn't exist.
        #    Or we can just do a basic histogram with no color distinction.
        chart = (
            alt.Chart(arrow_data)
            .mark_bar(opacity=0.6)
            .encode(
                x=alt.X(
                    f"{numeric_col}:Q",
                    bin=alt.Bin(maxbins=bins),
                    title=numeric_col
                ),
                y=alt.Y("count()", title="Frequency")
            )
            .properties(
                width=600,
                height=400,
                title=f"Histogram ({numeric_col} All)"
            )
            .configure_axisX(labelAngle=45)
        )

    return chart

# ------------------ Usage Example ------------------
if __name__ == "__main__":
    # Create some sample data
    df_eager = pl.DataFrame({
        "value": np.concatenate([np.random.normal(50, 10, 500), [120, 130]]),
    })
    # Mark outliers if value > 100
    df_eager = df_eager.with_columns(
        (pl.col("value") > 100).alias("is_outlier")
    )

    # Example 1: with outlier col
    chart1 = plot_histogram_optional_outliers(
        df_eager, numeric_col="value", outlier_col="is_outlier", bins=20
    )
    chart1.save("hist_with_outlier_toggle.html")

    # Example 2: no outlier col => no toggle
    df2 = pl.DataFrame({"value": np.random.normal(0, 1, 1000)})
    chart2 = plot_histogram_optional_outliers(df2, numeric_col="value", outlier_col=None, bins=20)
    chart2.display()


In [ ]:
import altair as alt
import polars as pl
import numpy as np

class Visualizer:
    def __init__(
        self,
        lazy_df: pl.LazyFrame,
        default_width: int = 800,
        default_height: int = 400,
        color_scheme: dict = None,
        theme: dict = None,
        debug: bool = False
    ):
        """
        Initialize the Visualizer with a Polars LazyFrame and optional settings.

        Args:
            lazy_df (pl.LazyFrame): The data source in lazy form.
            default_width (int): Default width for charts.
            default_height (int): Default height for charts.
            color_scheme (dict): A dictionary for color settings.
            theme (dict): A dictionary for overall chart theme settings.
            debug (bool): If True, prints additional debugging info.
        """
        self.lazy_df = lazy_df
        self.default_width = default_width
        self.default_height = default_height
        self.debug = debug

        # Default color scheme if not provided
        self.color_scheme = color_scheme or {
            "booking": "#1f77b4",
            "not_booking": "#d62728",
            "click": "#1f77b4",
            "not_click": "#ff7f0e"
        }

        # Default theme settings
        self.theme = theme or {
            "title_fontsize": 14,
            "title_anchor": "middle",
            "title_fontweight": "bold",
            "axis_label_angle": 45
        }

    def plot_categorical_distribution(self, var_x: str) -> alt.Chart:
        """
        Plots a categorical distribution (var_x) by click_bool and booking_bool,
        with interactive dropdowns for filtering. Data is collected from LazyFrame.
        """
        # Lazy operation: group & aggregate
        df_agg = (
            self.lazy_df
            .group_by([var_x, "click_bool", "booking_bool"])
            .agg(pl.count().alias("count_rows"))
        )

        # Materialize (collect) the result
        df_eager = df_agg.collect()

        # Compute proportion on the eager side
        n_rows = df_eager["count_rows"].sum()
        df_eager = df_eager.with_columns(
            (pl.col("count_rows") / n_rows * 100).alias("proportion")
        ).sort("proportion", descending=True)

        if self.debug:
            print(f"Aggregated DF for {var_x} (collected):\n{df_eager.head()}\n")

        # Build Altair param selections for interactive filtering
        click_selection = alt.param(
            name="click_bool_selection",
            bind=alt.binding_select(
                options=["All", 0, 1],
                labels=["Both", "Not Clicked", "Clicked"],
                name="Click Status"
            ),
            value="All"
        )

        book_selection = alt.param(
            name="book_bool_selection",
            bind=alt.binding_select(
                options=["All", 0, 1],
                labels=["Both", "Not Booked", "Booked"],
                name="Booking Status"
            ),
            value="All"
        )

        filter_condition = (
            ((click_selection == "All") |
             ((click_selection == 0) & (alt.datum.click_bool == 0)) |
             ((click_selection == 1) & (alt.datum.click_bool == 1))) &
            ((book_selection == "All") |
             ((book_selection == 0) & (alt.datum.booking_bool == 0)) |
             ((book_selection == 1) & (alt.datum.booking_bool == 1)))
        )

        # Title expression
        title_expr = alt.expr(
            f"click_bool_selection === 1 ? 'Distribution of {var_x} (Clicked)' : "
            f"click_bool_selection === 0 ? 'Distribution of {var_x} (Not Clicked)' : "
            f"'Distribution of {var_x} by Click & Booking Status'"
        )

        # Zoom
        zoom = alt.selection_interval(bind="scales")

        # Build the chart
        chart = (
            alt.Chart(df_eager.to_pandas())  # alt.Chart requires Pandas data
            .transform_calculate(
                click_label="datum.click_bool == 1 ? 'Clicked' : 'Not Clicked'",
                book_label="datum.booking_bool == 1 ? 'Booked' : 'Not Booked'"
            )
            .mark_bar(opacity=0.8)
            .encode(
                x=alt.X(f"{var_x}:O", title=var_x.replace('_', ' ').title()),
                xOffset=alt.XOffset("book_label:N"),
                y=alt.Y(
                    "proportion:Q",
                    title=f"Percentage of {var_x.replace('_', ' ').title()}",
                    scale=alt.Scale(domain=[0, 100]),
                    axis=alt.Axis(tickCount=10),
                ),
                color=alt.Color(
                    "book_label:N",
                    title="Booking Status",
                    scale=alt.Scale(
                        domain=["Not Booked", "Booked"],
                        range=[self.color_scheme["not_booking"], self.color_scheme["booking"]]
                    )
                ),
                tooltip=[
                    alt.Tooltip(var_x, title=var_x.replace("_", " ").title()),
                    alt.Tooltip("proportion", title="Percentage"),
                    alt.Tooltip("click_label:N", title="Click Status"),
                    alt.Tooltip("book_label:N", title="Booking Status")
                ]
            )
            .transform_filter(filter_condition)
            .add_params(click_selection, book_selection, zoom)
            .properties(
                width=self.default_width,
                height=self.default_height,
                title=alt.TitleParams(
                    text=title_expr,
                    fontSize=self.theme["title_fontsize"],
                    anchor=self.theme["title_anchor"],
                    fontWeight=self.theme["title_fontweight"]
                )
            )
        )

        return chart

    def plot_timeseries(self, datetime_col: str = "date") -> alt.Chart:
        """
        Builds a time-series chart of daily clicks vs. bookings, using a LazyFrame and collecting results.
        """
        # Lazy operation: ensure datetime col, then group/aggregate
        df_local = self.lazy_df
        # If there's no such column, maybe we look for time columns
        # But with lazy, we can do something like a conditional .with_columns
        # or you could do it outside. Here, let's assume the user has the correct col.

        df_agg = (
            df_local
            .group_by(datetime_col)
            .agg(
                pl.col("click_bool").sum().alias("click_count"),
                pl.col("booking_bool").sum().alias("book_count")
            )
            .sort(datetime_col)
        )

        # Collect to get an eager DataFrame
        df_eager = df_agg.collect()

        if df_eager.is_empty():
            raise ValueError(f"No data found after grouping by {datetime_col}.")

        if self.debug:
            print(f"Time-series aggregated (collected):\n{df_eager.head()}\n")

        # Melt for Altair
        melted_df = df_eager.melt(id_vars=[datetime_col], variable_name="metric", value_name="count")
        first_date, last_date = melted_df[datetime_col].min(), melted_df[datetime_col].max()

        # Selection param
        selection_param = alt.param(
            name="selection",
            bind=alt.binding_select(
                options=["Both", "click_count", "book_count"],
                labels=["Both", "Clicked", "Booked"],
                name="Filter"
            ),
            value="Both"
        )

        filter_condition = (
            (selection_param == "Both") |
            ((selection_param == "click_count") & (alt.datum.metric == "click_count")) |
            ((selection_param == "book_count") & (alt.datum.metric == "book_count"))
        )

        title_expr = alt.expr(
            "selection === 'click_count' ? 'Daily Click Count' : "
            "selection === 'book_count' ? 'Daily Booking Count' : "
            "'Daily Clicked vs Booked Count'"
        )

        zoom = alt.selection_interval(bind="scales")

        chart = (
            alt.Chart(melted_df.to_pandas())
            .mark_line(point=True)
            .encode(
                x=alt.X(
                    f"{datetime_col}:T",
                    title=datetime_col.replace("_", " ").title(),
                    scale=alt.Scale(domain=[first_date, last_date])
                ),
                y=alt.Y("count:Q", title="Count"),
                color=alt.Color(
                    "metric:N",
                    title="Metric",
                    scale=alt.Scale(
                        domain=["click_count", "book_count"],
                        range=[self.color_scheme["click"], self.color_scheme["not_click"]]
                    )
                ),
                tooltip=[
                    alt.Tooltip(datetime_col, title="Date"),
                    alt.Tooltip("metric:N", title="Metric"),
                    alt.Tooltip("count:Q", title="Count")
                ]
            )
            .transform_filter(filter_condition)
            .add_params(selection_param, zoom)
            .properties(
                width=self.default_width,
                height=self.default_height,
                title=alt.TitleParams(
                    text=title_expr,
                    fontSize=self.theme["title_fontsize"],
                    anchor=self.theme["title_anchor"],
                    fontWeight=self.theme["title_fontweight"]
                )
            )
            .configure_axisX(labelAngle=self.theme["axis_label_angle"])
        )

        return chart

    def plot_histogram(self) -> alt.Chart:
        """
        Demonstrates a histogram comparing two distributions.
        For a real scenario, you'd modify this to do lazy ops, then collect.
        Here we show an example with simulated data.
        """
        np.random.seed(42)
        original_data = np.random.normal(loc=50, scale=10, size=100).tolist() + [150, 160, 170]
        cleaned_data = [x for x in original_data if x < 100]

        # For demonstration, we just create a small eager DataFrame
        df_hist = pl.DataFrame({
            "value": original_data + cleaned_data,
            "dataset_type": ["Original (with outliers)"] * len(original_data) +
                            ["Cleaned (without outliers)"] * len(cleaned_data)
        })

        dataset_selection = alt.param(
            name="dataset_type",
            bind=alt.binding_select(
                options=["Both", "Original (with outliers)", "Cleaned (without outliers)"],
                labels=["Both", "Original", "Without Outliers"],
                name="Dataset"
            ),
            value="Both"
        )

        filter_condition = (
            (dataset_selection == "Both") |
            (alt.datum.dataset_type == dataset_selection)
        )

        title_expr = alt.expr(
            "dataset_type === 'Original (with outliers)' ? 'Original Distribution' : "
            "dataset_type === 'Cleaned (without outliers)' ? 'Post Process Distribution' : "
            "'Value Distribution with and without Outliers'"
        )

        chart = (
            alt.Chart(df_hist.to_pandas())
            .mark_bar(opacity=0.6)
            .encode(
                x=alt.X("value:Q", bin=alt.Bin(maxbins=30), title="Value Distribution"),
                y=alt.Y("count()", title="Frequency"),
                color=alt.Color(
                    "dataset_type:N",
                    title="Dataset Type",
                    scale=alt.Scale(
                        domain=["Original (with outliers)", "Cleaned (without outliers)"],
                        range=[self.color_scheme["not_booking"], self.color_scheme["booking"]]
                    )
                ),
                tooltip=["dataset_type", "value"],
                xOffset=alt.XOffset("dataset_type:N")
            )
            .transform_filter(filter_condition)
            .add_params(dataset_selection)
            .properties(
                width=int(self.default_width * 0.75),
                height=self.default_height,
                title=alt.TitleParams(
                    text=title_expr,
                    fontSize=self.theme["title_fontsize"],
                    anchor=self.theme["title_anchor"],
                    fontWeight=self.theme["title_fontweight"]
                )
            )
            .configure_axisX(labelAngle=self.theme["axis_label_angle"])
        )

        return chart

# ------------------------------------------------------------------------
# Example usage:

if __name__ == "__main__":
    # Suppose we have a Polars LazyFrame called lazy_df
    lazy_df_example = pl.DataFrame({
        "date": ["2025-01-01", "2025-01-02", "2025-01-03"] * 2,
        "click_bool": [1, 0, 1, 1, 0, 0],
        "booking_bool": [0, 0, 1, 0, 1, 0],
        "category_var": ["A", "B", "A", "B", "A", "B"]
    }).lazy()

    visualizer = Visualizer(lazy_df_example, debug=True)

    # Categorical distribution
    cat_chart = visualizer.plot_categorical_distribution("category_var")
    cat_chart.display()

    # Time series
    time_chart = visualizer.plot_timeseries("date")
    time_chart.display()

    # Histogram
    hist_chart = visualizer.plot_histogram()
    hist_chart.display()


In [ ]:
import polars as pl
import altair as alt
import numpy as np
from typing import List, Optional

class Visualizer:
    def __init__(self, df: pl.DataFrame, debug: bool = False):
        self.df = df
        self.debug = debug

    def _compute_histogram(
        self,
        column: str,
        bins: Optional[int] = None,
        normalize: bool = True
    ) -> pl.DataFrame:
        """Compute histogram for a numeric column."""
        
        # Get min/max values for the bin width
        min_val = self.df.select(pl.col(column).min()).item()
        max_val = self.df.select(pl.col(column).max()).item()
        
        # Use Sturges if bins not provided
        if bins is None:
            non_null_count = self.df.filter(pl.col(column).is_not_null()).shape[0]
            bins = int(1 + 3.322 * np.log10(non_null_count)) if non_null_count > 1 else 1
            print(f"For {column=} using Sturges rule: {bins=} bins.")
        bins = max(bins, 1)

        # Create bin edges and bin width
        bin_edges = np.linspace(min_val, max_val, bins + 1).tolist()
        bin_width = (max_val - min_val) / bins if bins > 0 else 1e-9

        # Complete the preprocessing for histogram
        hist_data = (
            self.df
            .select(pl.col(column))
            .filter(pl.col(column).is_not_null())
            .with_columns(
                pl.col(column).cut(bin_edges, include_breaks=True).alias("bin_range")
            )
            .group_by("bin_range")
            .agg(pl.len().alias("count"))
            .filter(pl.col("bin_range").is_not_null())
            .with_columns([
                (pl.col("bin_range").struct.field("breakpoint") - bin_width / 2).alias("bin_center"),
                pl.lit(column).alias("column"),
            ])
            .select(["column", "bin_center", "count"])
        )

        if normalize:
            hist_data = hist_data.with_columns(
                (pl.col("count") / pl.col("count").sum()).alias("rel_freq")
            )

        return hist_data

    def plot_histogram_with_checkboxes(
        self,
        columns: List[str],
        bins: int = 30,
        normalize: bool = True,
        width: int = 600,
        height: int = 400
    ) -> alt.Chart:
        """Dynamically allows selection of multiple columns to compare in a histogram using checkboxes."""
      


        # 📊 Get the data for the histogram
        frames = [self._compute_histogram(col, bins=bins, normalize=normalize) for col in columns]
        hist_data = pl.concat(frames, how="vertical")

        # ✅ Dynamically create checkboxes for all columns
        column_checkboxes = {
            col: alt.param(name=f"checkbox_{col}", bind=alt.binding_checkbox(), value=True)
            for col in columns
        }

        # 🔎 Build the filter condition using str **Altair Expression**
        filter_condition: str = " || ".join([f"checkbox_{col} && datum.column == '{col}'" for col in columns])
        

        chart = (
            alt.Chart(hist_data)
            .mark_bar(opacity=0.6)
            .encode(
                x=alt.X("bin_center:Q", title="Value"),
                y=alt.Y("rel_freq:Q" if normalize else "count:Q", title="Relative Frequency" if normalize else "Count"),
                color=alt.Color("column:N", title="Column"),
                tooltip=[
                    alt.Tooltip("column:N", title="Column"),
                    alt.Tooltip("bin_center:Q", title="Bin Center"),
                    alt.Tooltip("rel_freq:Q" if normalize else "count:Q", title="Frequency"),
                ]
            )
            .transform_filter(filter_condition)     # Filter expects a string condition
            .add_params(*column_checkboxes.values()) # Unpack the checkboxes
            .properties(
                width=width, height=height, title="Compare Multiple Columns (Checkbox Toggle)"
            )
            .configure_axisX(labelAngle=45)
        ).interactive()

        return chart


# ---------------- EXAMPLE USAGE ------------------
if __name__ == "__main__":
    
    # Create a sample dataset
    np.random.seed(42)
    train_df = pl.DataFrame({
        "price_usd": np.random.normal(100, 20, 1000),
        "srch_booking_window": np.random.randint(1, 30, 1000),
        "srch_adults_count": np.random.randint(1, 5, 1000)
    })
    viz = Visualizer(train_df, debug=True)
    chart = viz.plot_histogram_with_checkboxes(
        columns=["price_usd", "srch_booking_window", "srch_adults_count"],
        bins=None,
        normalize=True,
        width=800,
        height=400
    )

    chart.display()
    print("Saved 'hist_checkbox_columns.html'. Open in a browser to see the interactive chart.")


In [145]:
import re
import altair as alt
import polars as pl
import numpy as np
from typing import List, Optional


class Visualizer:
    def __init__(
        self,
        lazy_df: pl.LazyFrame,
        default_width: int = 800,
        default_height: int = 400,
        color_scheme: dict = None,
        theme: dict = None,
        debug: bool = False
    ):
        """
        🚀 Initialize the Visualizer with a Polars LazyFrame and optional settings.

        Args:
            lazy_df (pl.LazyFrame): The data source in lazy form.
            default_width (int): Default width for charts.
            default_height (int): Default height for charts.
            color_scheme (dict): A dictionary for color settings.
            theme (dict): A dictionary for overall chart theme settings.
            debug (bool): If True, prints additional debugging info.
        """
        self.lazy_df = lazy_df
        self.default_width = default_width
        self.default_height = default_height
        self.debug = debug

        # 🎨 Default color scheme
        self.color_scheme = color_scheme or {
            "booking": "#1f77b4",
            "not_booking": "#d62728",
            "click": "#1f77b4",
            "not_click": "#ff7f0e"
        }

        # 📏 Default theme settings
        self.theme = theme or {
            "title_fontsize": 14,
            "title_anchor": "middle",
            "title_fontweight": "bold",
            "axis_label_angle": 45
        }

    def _compute_histogram(
        self,
        column: str,
        bins: Optional[int] = None,
        normalize: bool = True
    ) -> pl.DataFrame:
        """📊 Compute histogram for a numeric column in a LazyFrame."""

        # 🏗️ Lazy operation: Get min/max
        min_max = self.lazy_df.select([
            pl.col(column).min().alias("min_val"),
            pl.col(column).max().alias("max_val")
        ]).collect()

        min_val, max_val = min_max.item(0,0), min_max.item(0,1)


        print(min_max, min_val, max_val)

        # 🧮 Use Sturges' formula if bins not provided
        if bins is None:
            non_null_count = self.lazy_df.select(pl.col(column).is_not_null().sum()).collect().item()
            bins = max(1, int(1 + 3.322 * np.log10(non_null_count))) if non_null_count > 1 else 1
            print(f"For {column=} using Sturges rule: {bins=} bins.")

        # 🔢 Define bin edges
        bin_edges = np.linspace(min_val, max_val, bins + 1).tolist()
        bin_width = (max_val - min_val) / bins if bins > 0 else 1e-9  # Avoid div by zero

        # 🛠️ Apply binning in LazyFrame
        hist_data = (
            self.lazy_df
            .select(pl.col(column))
            .filter(pl.col(column).is_not_null())
            .with_columns(
                pl.col(column).cut(bin_edges, include_breaks=True).alias("bin_range")
            )
            .group_by("bin_range")
            .agg(pl.len().alias("count"))
            .filter(pl.col("bin_range").is_not_null())
            .with_columns([
                (pl.col("bin_range").struct.field("breakpoint") - bin_width / 2).alias("bin_center"),
                pl.lit(column).alias("column"),
            ])
            .select(["column", "bin_center", "count"])
        )

        # 📉 Normalize if requested
        if normalize:
            hist_data = hist_data.with_columns(
                (pl.col("count") / pl.col("count").sum()).alias("rel_freq")
            )

        return hist_data.collect()  # Materialize LazyFrame at this point

    def plot_histogram_with_checkboxes(
        self, columns: List[str], bins: int = 30, normalize: bool = True, logy: bool = False, logx: bool = False
    ) -> alt.Chart:
        """📊 Allows selection of multiple columns to compare in a histogram using checkboxes."""

        # 📊 Compute histograms
        frames = [
    self._compute_histogram(col, bins=None, normalize=True)  # Compute per column
    for col in columns
]
        #print frames
        for frame in frames:
            print(frame)    

        hist_data = pl.concat(frames, how="vertical")

        # ✅ Create checkboxes dynamically
        column_checkboxes = {
            col: alt.param(name=f"checkbox_{col}", bind=alt.binding_checkbox(), value=True)
            for col in columns
        }

        # 🔎 Create filter expression
        combined_filter = " || ".join([f"checkbox_{col} && datum.column == '{col}'" for col in columns])

        # 🔥 Set Log Scale if needed
        y_scale = alt.Scale(type="symlog") if logy else alt.Undefined
        x_scale = alt.Scale(type="symlog") if logx else alt.Undefined
        title="Relative Frequency" if normalize else "Count"


        # 🎨 Build the histogram chart
        return (
            alt.Chart(hist_data)
            .mark_bar(opacity=0.6)
            .encode(
                x=alt.X("bin_center:Q", title="Value", scale=x_scale),
                y=alt.Y("rel_freq:Q" if normalize else "count:Q", title=title,scale=y_scale),
                color=alt.Color("column:N", title="Column"),
                tooltip=["column:N", "bin_center:Q", "rel_freq:Q" if normalize else "count:Q"]
            )
            .transform_filter(combined_filter)  # ✅ Ensure this exists
            .add_params(*column_checkboxes.values())
            .properties(width=self.default_width, height=self.default_height)
            .interactive()
        )


    def plot_conditional_barchart(self, var_x: str) -> alt.Chart:
        """📊 Categorical Distribution with Click & Booking Filters."""

        df_agg = (
            self.lazy_df.group_by([var_x, "click_bool", "booking_bool"])
            .agg(pl.count().alias("count_rows"))
        ).collect()

        n_rows = df_agg["count_rows"].sum()
        df_agg = df_agg.with_columns(
            (pl.col("count_rows") / n_rows * 100).alias("proportion")
        ).sort("proportion", descending=True)

        chart = (
        alt.Chart(df_agg)
        .mark_bar(opacity=0.8)
        .encode(
            x=alt.X(f"{var_x}:O", title=var_x.replace("_", " ").title()),  
            xOffset=alt.XOffset("booking_bool:N"),  # ✅ Keeps bars side by side (not stacked)
            y=alt.Y("proportion:Q", title="Percentage"),  
            color=alt.Color("booking_bool:N", title="Booking Status"),
            tooltip=[
                alt.Tooltip("category", title="Category"),
                alt.Tooltip("sum(proportion):Q", title="Total Percentage", format=".2f"),  # ✅ Ensures total proportion
                alt.Tooltip("booking_bool:N", title="Booking Status")  # ✅ Keeps tooltip consistent
            ]
        )
        .properties(width=self.default_width, height=self.default_height)
    )
        return chart

    def plot_timeseries(self, datetime_col: str = "date") -> alt.Chart:
        """📈 Time-Series Chart of Clicks vs. Bookings."""

        df_agg = (
            self.lazy_df
            .group_by(datetime_col)
            .agg(
                pl.col("click_bool").sum().alias("click_count"),
                pl.col("booking_bool").sum().alias("book_count")
            )
            .sort(datetime_col)
        ).collect()

        melted_df = df_agg.melt(id_vars=[datetime_col], variable_name="metric", value_name="count")

        return (
            alt.Chart(melted_df.to_arrow())
            .mark_line(point=True)
            .encode(
                x=alt.X(f"{datetime_col}:T", title="Date"),
                y=alt.Y("count:Q", title="Count"),
                color=alt.Color("metric:N", title="Metric"),
                tooltip=[alt.Tooltip(datetime_col), alt.Tooltip("metric:N"), alt.Tooltip("count:Q")]
            )
            .properties(width=self.default_width, height=self.default_height)
        )




In [ ]:
import re
import altair as alt
import polars as pl
import numpy as np
from typing import List, Optional


class Visualizer:
    def __init__(
        self,
        lazy_df: pl.LazyFrame,
        default_width: int = 800,
        default_height: int = 400,
        color_scheme: dict = None,
        theme: dict = None,
        debug: bool = False
    ):
        """
        🚀 Initialize the Visualizer with a Polars LazyFrame and optional settings.
        """
        self.lazy_df = lazy_df
        self.default_width = default_width
        self.default_height = default_height
        self.debug = debug

        self.color_scheme = color_scheme or {
            "booking": "#1f77b4",
            "not_booking": "#d62728",
            "click": "#1f77b4",
            "not_click": "#ff7f0e"
        }

        self.theme = theme or {
            "title_fontsize": 14,
            "title_anchor": "middle",
            "title_fontweight": "bold",
            "axis_label_angle": 45
        }

    def _compute_histogram(
        self,
        column: str,
        bins: Optional[int] = None,
        normalize: bool = True
    ) -> pl.DataFrame:
        """📊 Compute histogram for a numeric column in a LazyFrame."""
        min_max = self.lazy_df.select([
            pl.col(column).min().alias("min_val"),
            pl.col(column).max().alias("max_val")
        ]).collect()

        min_val, max_val = min_max.item(0,0), min_max.item(0,1)

        if bins is None:
            non_null_count = self.lazy_df.select(pl.col(column).is_not_null().sum()).collect().item()
            bins = max(1, int(1 + 3.322 * np.log10(non_null_count))) if non_null_count > 1 else 1
        
        bin_edges = np.linspace(min_val, max_val, bins + 1).tolist()
        bin_width = (max_val - min_val) / bins if bins > 0 else 1e-9

        hist_data = (
            self.lazy_df
            .select(pl.col(column))
            .filter(pl.col(column).is_not_null())
            .with_columns(
                pl.col(column).cut(bin_edges, include_breaks=True).alias("bin_range")
            )
            .group_by("bin_range")
            .agg(pl.len().alias("count"))
            .filter(pl.col("bin_range").is_not_null())
            .with_columns([
                (pl.col("bin_range").struct.field("breakpoint") - bin_width / 2).alias("bin_center"),
                pl.lit(column).alias("column"),
            ])
            .select(["column", "bin_center", "count"])
        )

        if normalize:
            hist_data = hist_data.with_columns(
                (pl.col("count") / pl.col("count").sum()).alias("rel_freq")
            )

        return hist_data.collect()

    def plot_histogram_with_checkboxes(
        self, columns: List[str], bins: int = 30, normalize: bool = True, logy: bool = False, logx: bool = False
    ) -> alt.Chart:
        """📊 Allows selection of multiple columns to compare in a histogram using checkboxes."""
        frames = [
            self._compute_histogram(col, bins=None, normalize=True)
            for col in columns
        ]
        hist_data = pl.concat(frames, how="vertical")

        column_checkboxes = {
            col: alt.param(name=f"checkbox_{col}", bind=alt.binding_checkbox(), value=True)
            for col in columns
        }

        combined_filter = " || ".join([f"checkbox_{col} && datum.column == '{col}'" for col in columns])

        y_scale = alt.Scale(type="symlog") if logy else alt.Undefined
        x_scale = alt.Scale(type="symlog") if logx else alt.Undefined
        title="Relative Frequency" if normalize else "Count"

        return (
            alt.Chart(hist_data)
            .mark_bar(opacity=0.6)
            .encode(
                x=alt.X("bin_center:Q", title="Value", scale=x_scale),
                y=alt.Y("rel_freq:Q" if normalize else "count:Q", title=title, scale=y_scale),
                color=alt.Color("column:N", title="Column"),
                tooltip=["column:N", "bin_center:Q", "rel_freq:Q" if normalize else "count:Q"]
            )
            .transform_filter(combined_filter)
            .add_params(*column_checkboxes.values())
            .properties(width=self.default_width, height=self.default_height)
            .interactive()
        )

    def plot_conditional_barchart(self, var_x: str) -> alt.Chart:
        """📊 Categorical Distribution with Click & Booking Filters."""
        df_agg = (
            self.lazy_df.group_by([var_x, "click_bool", "booking_bool"])
            .agg(pl.count().alias("count_rows"))
        ).collect()

        n_rows = df_agg["count_rows"].sum()
        df_agg = df_agg.with_columns(
            (pl.col("count_rows") / n_rows * 100).alias("proportion")
        ).sort("proportion", descending=True)

        return (
            alt.Chart(df_agg)
            .mark_bar(opacity=0.8)
            .encode(
                x=alt.X(f"{var_x}:O", title=var_x.replace("_", " ").title()),  
                xOffset=alt.XOffset("booking_bool:N"),
                y=alt.Y("proportion:Q", title="Percentage"),  
                color=alt.Color("booking_bool:N", title="Booking Status"),
                tooltip=["category", "sum(proportion):Q", "booking_bool:N"]
            )
            .properties(width=self.default_width, height=self.default_height)
        )

# Example Usage
if __name__ == "__main__":
    lazy_df = pl.DataFrame({
        "price_usd": np.random.normal(100, 20, 1000),
        "srch_booking_window": np.random.randint(1, 30, 1000),
    }).lazy()
    viz = Visualizer(lazy_df)
    hist_chart = viz.plot_histogram_with_checkboxes(["price_usd", "srch_booking_window"], bins=20, normalize=True)
    hist_chart.display()


ColumnNotFoundError: price_usd

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'select' <---
DF ["date", "click_bool", "booking_bool", "category_var"]; PROJECT */4 COLUMNS

In [131]:
import altair as alt
import pandas as pd

df = pd.DataFrame(
[['L1', 2000],
 ['L2', 0],
 ['L3', 0],
 ['L4', 3000],
 ['L5', 101],
 ['L6', 100],
 ['L7', 99],
 ['L8', 250],
 ['L9', 770000]],
columns=['group', 'foos'])

chart = alt.Chart(df)

alt.Chart(df).transform_filter(
    alt.datum.foos > 0  
).mark_bar().encode(
    alt.X('foos', scale=alt.Scale(type='symlog')),
    y='group'
)

alt.Chart(...)